In [ ]:
import pandas as pd
import numpy as np
import scipy 
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

games = pd.read_csv('/datasets/games.csv')
games.columns = games.columns.str.lower()
games['year_of_release'] = games['year_of_release'].fillna(0)
games['year_of_release'] = games['year_of_release'].astype(int)
games['user_score'] = games['user_score'].replace('tbd', np.nan)
duplicates = games.duplicated()
duplicate_count = games.duplicated().sum()
print(duplicate_count)
print()
print(games)
print()
print(games.dtypes)

#Starting out I am importing extensions I feel like I will need to use for this analysis then putting the information into a dataframe. From there I use lower function to put every column header in lower case. The only column type I am changing is the year of release I am changing that to an integer to get rid of the decimal, I filled the NA with 0 before I did that. I am going to leave the scores and rating columns that do not have values as NaN, I do not want to switch them to 0 because that can mess with the averages of numbers later on in the analysis. I changed the values with TBD in user score to blank, since they technically havent been rated I did not want to fill it with 0 and I didnt want TBD written out for some columns.

I believe there is missing values in ratings and scores because the games were probably small or unpopular and did not get enough attention to warrant a rating or score.

In [ ]:
games['total_sales'] = (games['na_sales'] + games['eu_sales'] + games['jp_sales'] + games['other_sales'])
print(games)

In [ ]:
filtered_games = games[games['year_of_release'] != 0]
released_games_per_year = filtered_games['year_of_release'].value_counts().sort_index()

plt.figure(figsize=(12,6))
plt.bar(released_games_per_year.index, released_games_per_year.values, color='black')
plt.title('Number of Games Released Each Year(excludes blank release dates)', fontsize=14)
plt.xlabel('Year of Release', fontsize=12)
plt.ylabel('Number of Games', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


#Created a dataframe that excludes games missing data on release year. Then summed the values for each year, reset the index and printed it to a graph.

#It appears the most popular time for releasing games was the end of 2000s decade and since then the numbers have fallen off drastically. The numbers peaked in 2008 and 2009 when there was around 1400 games released each year but has fallen down to below 600 now in 2016.

In [ ]:
platform_sales = (filtered_games.groupby('platform')['total_sales'].sum().sort_values(ascending=False))
top_10_platforms = platform_sales.head(10).index
top_platforms_data = filtered_games[filtered_games['platform'].isin(top_10_platforms)]

sales_by_platform_year = top_platforms_data.pivot_table(index='year_of_release', columns='platform', values='total_sales', aggfunc='sum', fill_value=0)

plt.figure(figsize=(15, 8))
sales_by_platform_year.plot(kind='bar', stacked=True, figsize=(15,8))

plt.title('Total Sales by Platform by Year', fontsize=16)
plt.xlabel('Year of Release', fontsize=14)
plt.ylabel('Total Sales (in millions)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Platform', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()  
plt.show()

#Judging from the data of the platforms with the 10 highest sales I would say on average a platform has about 9 years of use. Most peak in usage about 3 to 4 years in then from there the number of games being released tends to fall off. The GameBoy looks like the only one to survive for over 15 years, going from 1985 and still having games released for it in 2001.

#I am going to take data from 2002 onward and 2012 onwardgiving me two data sets to reference. Before 2002 video games were not nearly as popular as they are now and all of the games released on those platforms are drastically different than the videog games being released now. From 2002 onward I will be able to analyze the large growth from 2004 to 2009 and also the decline after that. The data from 2012 onward will only include new data which I think will be the best for forecasting out in the future since those games and platforms will be the most similar to the ones being released now.

#For the first question on what platforms are leading in sales and which ones are growing and shrinking I am only going to use data from the last 5 years. This will allow me to see which platforms are currently on the growth trend and which ones are currently declining.

In [ ]:
recent_games = filtered_games[filtered_games['year_of_release'] > 2011]
top_platforms = (recent_games.groupby('platform')['total_sales'].sum().sort_values(ascending=False))
top_10_platforms_recent = top_platforms.head(10).index
recent_top_10_platforms = recent_games[recent_games['platform'].isin(top_10_platforms_recent)]

sales_by_platform_year_recent = recent_top_10_platforms.pivot_table(index='year_of_release', columns='platform', values='total_sales', aggfunc='sum', fill_value=0)
plt.figure(figsize=(15, 8))
sales_by_platform_year_recent.plot(kind='bar', stacked=True, figsize=(15,8))

plt.title('Total Sales by Platform by Year', fontsize=16)
plt.xlabel('Year of Release', fontsize=14)
plt.ylabel('Total Sales (in millions)', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Platform', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()  
plt.show()

In [ ]:
last_three_years = recent_games[recent_games['year_of_release'].isin([2014, 2015, 2016])]

games_count_by_year = last_three_years.groupby(['platform', 'year_of_release'])['name'].count().unstack(fill_value=0)
games_count_by_year['growth'] = games_count_by_year[2016] - games_count_by_year[2015]
games_count_by_year['growth_1'] = games_count_by_year[2015] - games_count_by_year[2014]
platform_growth = games_count_by_year['growth'].sort_values(ascending=False)
platform_growth = platform_growth.reset_index()
print(platform_growth)

platform_growth_1 = games_count_by_year['growth_1'].sort_values(ascending=False)
platform_growth_1 = platform_growth_1.reset_index()
print(platform_growth_1)

#The platforms with the most sales recently are the PS4, the XboxOne and the 3DS in that order. The platforms with the most growth in releasing games over the last year are the PS4, XboxOne then the PC in that order. With the largest sales amount and most growth in games being released on it the PS4 looks like the best option but I would be more interested in seeing which consoles are the most popular and currently have the most sales. That would allow us to see where there is potential. It is a possibility that since the PS4 has the most games being sold on it and most growth in games the market could be saturated and their could be to much competition.

In [ ]:
plt.figure(figsize=(15, 8))
games.boxplot(column='total_sales', by='platform', vert=False, grid=False, showfliers=False)
plt.title('Global Sales Distribution by Platform', fontsize=16)
plt.xlabel('Global Sales (in millions)', fontsize=14)
plt.ylabel('Platform', fontsize=14)
plt.suptitle('')  # Remove the automatic subplot title
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
games_per_platform = games.groupby('platform')['name'].count().sort_values(ascending=False)

plt.figure(figsize=(15, 8))
games_per_platform.plot(kind='bar')
plt.title('Total Number of Games Released per Platform', fontsize=16)
plt.xlabel('Platform', fontsize=14)
plt.ylabel('Number of Games Released', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
highest_selling_games = games.loc[games.groupby('platform')['total_sales'].idxmax()]
highest_selling_games = highest_selling_games.sort_values(by='total_sales', ascending=False)
plt.figure(figsize=(15, 8))
bars = plt.bar(highest_selling_games['platform'], highest_selling_games['total_sales'])
for bar, game_name in zip(bars, highest_selling_games['name']):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.5,
        game_name,
        ha='center',
        va='bottom',
        fontsize=9,
        rotation=90
    )

plt.title('Highest-Selling Game for Each Platform', fontsize=16)
plt.xlabel('Platform', fontsize=14)
plt.ylabel('Global Sales (in millions)', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

There is substantial differences in sales between platforms. The platforms that show a higher median tend to be the ones with fewer games released and a few extremely popular high selling games. NES and GameBoy are perfect examples of this of the highest selling games of all time per platform the are #2 and #3 on the list and both have less than 100 games ever released. This leads to wide variation in both of their box plots.

#For the code I first created a box plot to get an idea of the median sales for each platform along with the variation of the sales numbers for their games. Once I saw that I was curious which platforms had sold the most games so I plotted total sales per platform to get an idea for which ones had lower values which would cause more variation. Once I saw that data I was curious what the highest selling game was on each platform because some of the box plots were far wider than others. This allowed me to see which platforms did not release very many games and also which platforms had games that sold extremely well.

In [ ]:
ps4_games = games[games['platform'] == 'PS4'].copy()
ps4_games['user_score'] = pd.to_numeric(ps4_games['user_score'], errors='coerce')
ps4_games['critic_score'] = pd.to_numeric(ps4_games['critic_score'], errors='coerce')
ps4_games['normalized_critic_score'] = ps4_games['critic_score'] / 10
ps4_games['average_score'] = (ps4_games['user_score'] + ps4_games['normalized_critic_score'] / 2)

#Building scatterplot of user score to sales
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.scatter(ps4_games['user_score'], ps4_games['total_sales'], alpha=0.5)
plt.title('User Reviews vs. Sales (PS4)')
plt.xlabel('User Score')
plt.ylabel('Total Sales ($M)')

#Building scatterplot of critic score to sales
plt.subplot(1, 3, 2)
plt.scatter(ps4_games['critic_score'], ps4_games['total_sales'], alpha=0.5)
plt.title('Critic Reviews vs. Sales (PS4)')
plt.xlabel('Critic Score')
plt.ylabel('Total Sales ($M)')

#Building scatterplot of average score of user and critic score to sales
plt.subplot(1, 3, 3)
plt.scatter(ps4_games['average_score'], ps4_games['total_sales'], alpha=0.5)
plt.title('Average Reviews vs. Sales (PS4)')
plt.xlabel('Average Score')
plt.ylabel('Total Sales ($M)')
plt.tight_layout()
plt.show()

# Calculate correlation for User Reviews vs. Sales
user_score_corr = ps4_games[['user_score', 'total_sales']].corr()
print(f"Correlation between User Score and Sales: {user_score_corr}")

# Calculate correlation for Professional Reviews vs. Sales
critic_score_corr = ps4_games[['critic_score', 'total_sales']].corr()
print(f"Correlation between Critic Score and Sales: {critic_score_corr}")

# Calculate correlation for Average Reviews vs. Sales
average_score_corr = ps4_games[['average_score', 'total_sales']].corr()
print(f"Correlation between Average Score and Sales: {average_score_corr}")

#For user scores there is basically 0 correlation between user scores and sales, for critic scores there is an average correlation of about 0.41 between critic scores and sales and for the average of the two reviews combined there is little to no correlation between reviews and scores with a correlation of 0.11. The reasoning behind this data could be because critics tend to know which games are going to be better since their whole career is based off playing and judging video games. While the average player just goes about playing games he enjoys and has a passion for.

In [ ]:
# Group by 'name' to count how many platforms each game appears on
games_platform_count = games.groupby('name')['platform'].nunique()

# Filter games that are on multiple platforms
multi_platform_games = games_platform_count[games_platform_count > 1].index

# Filter the dataset to include only games on multiple platforms
multi_platform_sales = games[games['name'].isin(multi_platform_games)]

# Aggregate total sales across all platforms for each game
aggregated_sales = multi_platform_sales.groupby('name')['total_sales'].sum()

# Sort by total sales in descending order
top_10_multi_platform = aggregated_sales.sort_values(ascending=False).head(10)

# Display the top 10 highest-selling games on multiple platforms
print(top_10_multi_platform)

In [ ]:
print(multi_platform_sales)
#Getting a visual of the data here

In [ ]:
# Step 1: Filter the dataset for the top 10 multi-platform games
top_10_games = top_10_multi_platform.index  # These are the top 10 games
filtered_data = multi_platform_sales[multi_platform_sales['name'].isin(top_10_games)]

# Step 2: Group by game name and platform, summing total sales
platform_comparison = filtered_data.groupby(['name', 'platform'], as_index=False)['total_sales'].sum()

# Step 3: Compute total sales for each game and sort them
game_sales_total = platform_comparison.groupby('name')['total_sales'].sum()
sorted_game_sales = game_sales_total.sort_values(ascending=False)

# Update games_unique with the sorted order
games_unique = sorted_game_sales.index.tolist()

# Get unique platforms
platforms_unique = platform_comparison['platform'].unique()

# Step 4: Prepare the data for plotting
bar_width = 0.8
x_indexes = np.arange(len(games_unique))  # X positions for bars
bottom_values = np.zeros(len(games_unique))  # To stack bars

# Create a figure for the plot
plt.figure(figsize=(15, 8))

# Step 5: Plot data for each platform
for platform in platforms_unique:
    # Filter the data for the current platform
    platform_data = platform_comparison[platform_comparison['platform'].isin([platform])]
    
    # Align sales with the corresponding games (now sorted)
    sales = [
        platform_data[platform_data['name'] == game]['total_sales'].sum() 
        for game in games_unique
    ]
    
    # Plot the bar for this platform
    plt.bar(x_indexes, sales, width=bar_width, bottom=bottom_values, label=platform)
    
    # Update the bottom values for stacking
    bottom_values += sales

# Step 6: Customize the plot
plt.title('Total Sales by Top 10 Multi-Platform Games', fontsize=16)
plt.xlabel('Game Name', fontsize=12)
plt.ylabel('Total Sales (millions)', fontsize=12)
plt.xticks(ticks=x_indexes, labels=games_unique, rotation=45, ha='right')
plt.legend(title='Platform', fontsize=10, loc='upper left')

# Step 7: Show the plot
plt.tight_layout(pad=3)
plt.show()


#I looked at the ten highest selling games that were on multiple platforms. Sales across the platforms for the call of duty games seem fairly similar between the playstation, xbox and PC. The games that were only on GameBoy and NES were drastically different though. Super Mario bros mainly had sales on the NES while Tetris mainly had sales on the GameBoy. The Call of Duty games sales between platforms probably depends a lot more on the total sales of each consoles. While the difference in sales for Super Mario Bros and Tetris probably depends on which platform is more enjoyeable to play the game on.

In [ ]:
genre_sales = filtered_games.groupby('genre')['total_sales'].sum().sort_values(ascending=False)
genre_count = filtered_games.groupby('genre')['name'].count().sort_values(ascending=False)
genre_avg_sales = filtered_games.groupby('genre')['total_sales'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
genre_count.plot.bar(color='black', alpha=0.7)
plt.title('Total Games by Genre')
plt.xlabel('Genre')
plt.ylabel('Total Games')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
genre_sales.plot.bar(color='black', alpha=0.7)
plt.title('Total Sales by Genre')
plt.xlabel('Genre')
plt.ylabel('Total Sales (in millions)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
genre_avg_sales.plot.bar(color='black', alpha=0.7)
plt.title('Average Sales by Genre')
plt.xlabel('Genre')
plt.ylabel('Average Sales (in millions)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


scatter_data = pd.DataFrame({
    'total_games': genre_count,
    'avg_sales': genre_avg_sales
    }).reset_index()

for i, genre in enumerate(scatter_data['genre']):
    plt.text(
        scatter_data['total_games'][i],
        scatter_data['avg_sales'][i] + 0.01,
        genre,
        fontsize=9,
        ha='center'
    )
                                                 
x = scatter_data['total_games']
y = scatter_data['avg_sales']
slope, intercept = np.polyfit(x, y, 1)
best_fit_line = slope * x + intercept

plt.figure(figsize=(10, 6))
plt.scatter(x, y, color='black', alpha=0.7)
plt.plot(x, best_fit_line, color='red', linestyle='--', label='Best Fit Line')
plt.title('Scatterplt of Avg. Sales vs. Total Games')
plt.xlabel('Total Games Released')
plt.ylabel('Average Sales (millions)')
plt.grid(True)
plt.tight_layout()
plt.show()

#Looking at the data it seems the average sales correlates with total number of games made for the genre. I am assuming that is because video game developers try to create games in the categories that are likely to make the most revenue since they have the best chance of making money. Of course the genres with the most games also tend to have the most sales. The more games you release the more revenue you can generate. The 3rd, 4th and 5th genres with the most games generate the 2nd, 3rd and 1st best average sales per game.

In [ ]:
regions = ['na_sales', 'eu_sales', 'jp_sales']

for region in regions:
    top_platforms = (
    filtered_games.groupby('platform')[region].sum().sort_values(ascending=False).head(5))
    
    plt.figure(figsize=(8, 5))
    top_platforms.plot(kind='bar')
    plt.title(f'Top 5 Platforms in {region.upper()}')
    plt.xlabel('Platform')
    plt.ylabel(f'Total Sales in {region.upper()} (in millions)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    
plt.show()


#North America and Europe have the same exact top 4 platforms in sales. Both continents have PS2, PS3, Xbox 360 and the Wii in their top 4. Japan's sales are very different with the Nintendo DS leading the way then the Playstation, then PS2 then Super Nintendo Entertainment System then the 3DS. Japan's platforms sales are heavily favored towards more Nintendo products, since Nintendo was founded in Japan it is much more popular in that culture. Sony (who makes Playstation) has 2 platforms in the top 5 is also an Japanese company. It is interesting the Wii did not break into Japan's top platforms given the immense popularity of the product in the rest of the world. Two of Japan's top 5 highest platforms and its platforms with the most sales were portable gaming platforms (DS & 3DS). While North America only had one handheld platform in its top 5 and it was 5th (DS). Europe did not have any handheld platforms in its top 5.

In [ ]:
regions = ['na_sales', 'eu_sales', 'jp_sales']

for region in regions:
    top_genres = (
    filtered_games.groupby('genre')[region].sum().sort_values(ascending=False).head(5))
    
    plt.figure(figsize=(8, 5))
    top_platforms.plot(kind='bar')
    plt.title(f'Top 5 Genres in {region.upper()}')
    plt.xlabel('Genre')
    plt.ylabel(f'Total Sales in {region.upper()} (in millions)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    
plt.show()

#Just like with the platforms the genres for the regions were similar for Europe and North America, with 4 of their top 5 being the same and there top 3 being indentical in sales. Action genre was the most popular for both of them by a decent margin. For Japan 'Role Playing' was by the far the highest selling genre, over 2x more sales in that genre than the 2nd most popular one. Role-playing was not even in the top 5 for the other two regions. Different games are tailored for different platforms, therefore it is not surprising to see the genre lists be different since the platform lists were. The handheld DS was made to favor more role playing games while consoles like Playstation, Xbox and Wii favor action and sports games.

In [ ]:
for region in regions:
    esrb_sales = (
    filtered_games.groupby('rating')[region].sum().reset_index().sort_values(by=region, ascending=False)
    ) 
    
    esrb_game_count = (
    filtered_games.groupby('rating')['name'].count().reset_index().rename(columns={'name': 'game_count'}).sort_values(by='game_count', ascending=False)
    ) 
    
    merged_data = pd.merge(esrb_sales, esrb_game_count, on='rating')
    
    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax1.scatter(merged_data['rating'], merged_data[region], s=100, label='Total_Sales')
    ax1.set_xlabel('ESRB Rating')
    ax1.set_ylabel(f'Sales in {region.upper()} (in millions)')
    ax1.grid(True)
    
    for i, rating in enumerate(merged_data['rating']):
        ax1.text(rating, merged_data[region][i] + 0.2, str(round(merged_data[region][i], 2)),
                fontsize=9, ha='center')
        
    ax2 = ax1.twinx()
    ax2.bar(merged_data['rating'], merged_data['game_count'], alpha=0.5, color='orange', label='Game Count')
    ax2.set_ylabel('Number of Games', color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')
    
    plt.title(f'{region.upper()} Sales and Game counts by ESRB Rating')
    plt.tight_layout()
    
    
plt.show()


#For the most part the ratings with most sales goes E, T, M then E10+. Which makes sense because for the most part those are the rating with the most games being released. The outlier is Europe sales where there was about half as many mature games released as Teen games but mature games sales were about $60M higher.

In [ ]:
xone_scores = games[(games['platform'] == 'XOne') & (games['user_score'].notna())]['user_score']
pc_scores = games[(games['platform'] == 'PC') & (games['user_score'].notna())]['user_score']

xone_scores = pd.to_numeric(xone_scores, errors='coerce').dropna()
pc_scores = pd.to_numeric(pc_scores, errors='coerce').dropna()

t_stat, p_value = ttest_ind(xone_scores, pc_scores, equal_var=False)

print("Xbox One User Scores:\n", xone_scores.describe())
print("\nPC User Scores:\n", pc_scores.describe())
print(f"\nT-Statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("n\nReject the null hypothesis: The average user ratings for Xbox One and PC are significantly different.")
else:
    print("\nFail to reject the null hypothesis: There is no significant difference in average user ratings for Xbox One and PC.")
    

#The average score for XOne games was 6.52 while the average score for PC games was 7.06. Based off an alpha of 0.05 the null hypothesis can be rejected, the average user ratings between the platforms are significantly different. I chose the significane level of 0.05 because it is a happy medium between minimizing the risk of a Type I error (incorrectly rejecting the null hypothesis) while still having a reasonable chance of detecting a real effect. The hypothesis were based off whether or not the ratings were statistically different.


In [ ]:
action_scores = games[(games['genre'] == 'Action') & (games['user_score'].notna())]['user_score']
sports_scores = games[(games['genre'] == 'Sports') & (games['user_score'].notna())]['user_score']

action_scores = pd.to_numeric(action_scores, errors='coerce').dropna()
sports_scores = pd.to_numeric(sports_scores, errors='coerce').dropna()

t_stat, p_value = ttest_ind(action_scores, sports_scores, equal_var=False)

print("Action User Scores:\n", action_scores.describe())
print("\nSports User Scores:\n", sports_scores.describe())
print(f"\nT-Statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("n\nReject the null hypothesis: The average user ratings for the action and sports genres are significantly different.")
else:
    print("\nFail to reject the null hypothesis: There is no significant difference in average user ratings for the action and sports.")
    

#Based off our ttest the two genres do not have a statisically different user rating. The null hypothesis is based off there being no difference while the alternative hypothesis is based off there being a statiscally significant difference. I chose the significane level of 0.05 because it is a happy medium between minimizing the risk of a Type I error (incorrectly rejecting the null hypothesis) while still having a reasonable chance of detecting a real effect.